In [1]:
import pandas as pd
#import sqlite3 
import urllib.request
import warnings
warnings.filterwarnings('ignore')

changes made 6/1:
- moved the fillna stuff to map_values b/c it gets called before case_matcher
- deleted line about loading in unedited detail codes
- added judges_court as a parameter to case_matcher just for clarity
- dropped "Court Name" column b/c "Court" is there already
- merged just on judge and not both judge and year

In [2]:
#read in courts and judges
court_list = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/courts.csv")
judges = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/judges.csv")
#dictionary for replacing the court id with the court name
court_code = dict(court_list[["Court ID", "Court Name"]].values)
#codes=pd.read_csv("/Users/hinaljajal/Downloads/State-Sentencing-Project/CSV Processing/edited_codes.csv")

In [3]:
#Replace codes in the dataframe with their actual meaning
#Ex. replace "W" with "White" for defendant race
def map_values(df):
    #Fill in NaN values
    df['Race'] = df['Race'].fillna("U") #U is the code for unknowns 
    df['Sentence Y'] = df['Sentence Y'].fillna(0)
    df['Sentence M'] = df['Sentence M'].fillna(0)
    df['Sentence D'] = df['Sentence D'].fillna(0)
    df['Probation D'] = df['Probation D'].fillna(0)
    df['Probation Y'] = df['Probation Y'].fillna(0)
    df['Probation M'] = df['Probation M'].fillna(0)
    df["Charge Class"] = df['Charge Class'].fillna("Un")
    df['Judge'] = df['Judge'].fillna("N/A") #Nans are filled with unknowns 
    
    #f = open("/Users/hinaljajal/Downloads/State-Sentencing-Project/CSV Processing/edited_codes.csv")
    f = open(r"C:\Users\chris\Documents\GitHub\State-Sentencing-Project\CSV Processing/edited_codes.csv")
    for line in f:
        line = line.strip('\n')
        line = line.split(",")
        df = df.replace({line[2]:{line[0]:line[1]}})
    f.close()
    
    return df

In [4]:
def case_matcher(cases, judges_court):
    """
    cases: a dataframe of cases for 1 specifc court
    judges_court: list of judges for specific court
    """

    cases["Year"]=cases["Last Hearing Date"].str[-4:]

    
    
    #Merges the columns by the judge and the hearing year
    cases['Year']=cases['Year'].astype(int)
    cases['Judge']=cases['Judge'].astype(str)
    judges_court['Year']=judges_court['Year'].astype(int)
    judges_court['Judge']=judges_court['Judge'].astype(str)

    cases = cases.merge(judges_court, how="left",on = ["Judge"])

    #dropping the name and second court name column
    cases=cases.drop(['Name','Court Name'], axis=1)
    
    #Checking merge - can remove when submitting
    unknowns = cases["Judge Full Name"].isna().mean()
    if unknowns > 0.1:
        print(cases["Court"].iloc[1],":", (cases["Judge"]=="N/A").mean(), unknowns)
    
    #Saves the dataframe to a csv
#     file_name = '/Users/hinaljajal/Downloads/{}.csv'.format(court)
#     cases.to_csv(file_name, index = False)
    
    return cases

In [5]:
separate_court_data = []
for court in court_list["Court ID"]:
    try:
        #data = pd.read_csv("/Users/hinaljajal/Downloads/finished_courts/{}.csv".format(court))
        data = pd.read_csv("C:/Users/chris/Documents/case data/finished courts 5.29.4.30/{}.csv".format(court))
    except FileNotFoundError:
        continue
    data = map_values(data)
    #Slices the judges for the particular court 
    judges_court = judges[judges["Court Name"]==court_code[court]]
    #replace court code with name, ie "001C" with "Accomack"
    data["Court"] = court_code[court]
    data = case_matcher(data, judges_court)
    separate_court_data.append(data) #add to list for concatenation
    
final_cases = pd.concat(separate_court_data, ignore_index = True)

Augusta : 0.18791946308724833 0.3446966542874748
Bath : 0.0023346303501945525 1.0
Buckingham : 0.1813912333242581 0.26585897086849986
Campbell : 0.522095540482032 0.522239861451869
Charles City : 0.5366619115549215 0.5378031383737518
Charlottesville : 0.407513640578835 0.4430114947486467
Clifton Forge : 1.0 1.0
Essex : 0.9996096799375488 1.0
Floyd : 0.9530218384966989 0.9659725749111224
Fredericksburg : 0.8655533540900084 0.8764223039909426
Gloucester : 0.00491943533437901 0.1702314748799848
Goochland : 0.9989146164978292 1.0
Grayson : 0.9997834091401343 1.0
Greene : 0.999731255038968 1.0
Greensville : 0.005437763838870471 0.15175654081900355
Hanover : 0.003736647967417198 0.13979289940828402
King and Queen : 0.3553634977122522 0.3601931875953228
Lancaster : 0.46032048836322015 0.4618466234261732
Louisa : 0.5891704227067852 0.5892313314654647
Madison : 0.0006996268656716418 1.0
Martinsville : 0.5028108579426814 0.5047383033890915
Mathews : 0.14866911923826012 0.16771261631681456
Middle

In [7]:
final_cases["Judge Full Name"].isna().mean()

0.027194399003681775

Outlier courts:
- Madison - Uses first 4 letters of last name instead of judge initials (possibly? not certain)
- Pulaski and Surry - Majority uses ^ but also some cases have initials
- Bath - Last name, also WETS = John E. Wetsel?
- Buckingham - uses both
- Buena Vista - look into same name multiple initials
- Gloucester - just a mess with typos/judges outside of court
- Greensville - uses last name, initials, first letter of last name
- Hanover - ID is 1, 2, 3, instead of names